# Notebook to fetch data from opendota

In [1]:
%%time
import dota2api

#for rest api usage
import requests

#for pgadmin
import psycopg2

import time


CPU times: user 75.9 ms, sys: 37.2 ms, total: 113 ms
Wall time: 127 ms


In [2]:
#progress bar widget https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [3]:
#Postgres connection
hostname = 'localhost'
username = 'postgres'
password = ''
database = 'dota2_data'


# Matches

Found around 47k matches in the matches database..less then what I was looking for

###update 
use "on conflict do nothing" clause to keep udpating the local db
##insert into matches (match_id) values (35811738) on conflict (match_id) DO NOTHING


June 29th: 47600 matches 

July 1st: 48000 matches

In [4]:
%%time
#use requests to get data
endpoint='https://api.opendota.com/api/explorer?sql='
max_records = 48000
step = 400

query='select * from matches order by start_time desc LIMIT ' + str(step) + ' Offset '
query='select match_id, start_time, radiant_win, radiant_team_id, dire_team_id from matches order by start_time desc LIMIT ' + str(step) + ' Offset '

myConnection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
cur = myConnection.cursor()

#get 1 records at a time
for i in log_progress(range(max_records/step)):
    #print i, 'out of', max_records/step, 
    offset = i * step
    url = endpoint+query+str(offset)
    start = time.time()
    response = requests.get(url)
    end = time.time()
    #print 'got response in: ', (end-start)
    tuples = []
    insert_query = 'INSERT INTO matches ('

    #parse data and insert
    for j in range(step):
        #create the query
        values = ()
        try:
            for k, v in response.json()['rows'][j].iteritems():
                if k=='chat' or k=='picks_bans' or k=='objectives' or\
                    k=='radiant_gold_adv' or k=='radiant_xp_adv' or k=='teamfights'or k=='cosmetics':
                    continue
                if j==0:
                    insert_query+=' '+k+','

                values = values + (v,)
        except:
            pass
        tuples.append(values)
            
    insert_query = insert_query.rstrip(',')
    insert_query += ')' + ' values '
#     args_str = ','.join(cur.mogrify('(%s, %s, %s, %s, %s, %s, %s, '\
#         '%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)',x)\
#         for x in tuples)
    try:
        args_str = ','.join(cur.mogrify('(%s, %s, %s, %s, %s) ',x)\
            for x in tuples)
        cur.execute(insert_query + args_str+ " on conflict (match_id) DO UPDATE set start_time = EXCLUDED.start_time " )
    except Exception as e:
        pass
myConnection.commit()


cur.execute('select count(*) from table where guid = %s;',[guid])
rows = cur.fetchall()
print 'ResultCount = %d' % len(rows)

myConnection.close()    
    

CPU times: user 1min 35s, sys: 375 ms, total: 1min 35s
Wall time: 2min 16s


120 record insert speed:
step 5: 7.29 seconds
setp 10: 6.49 seconds
step 15: 5.9 seconds
step 20: 7 seconds
step 40: 11.2 seconds


just with 2 columns, took 2 seconds with step=10

# Player_matches

In [6]:
#%%time
#cols to obtain: "match_id":17955123,"account_id":89782335,"player_slot":0,"hero_id":31,
#use requests to get data
endpoint='https://api.opendota.com/api/explorer?sql='
max_records = 480000
step = 400

query='select match_id, hero_id, account_id, player_slot from player_matches LIMIT ' + str(step) + ' Offset '

myConnection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
cur = myConnection.cursor()

#get 1 records at a time
for i in log_progress(range(max_records/step)):
    #print i, 'out of', max_records/step, 
    offset = i * step
    url = endpoint+query+str(offset)
    start = time.time()
    response = requests.get(url)
    end = time.time()
    tuples = []
    insert_query = 'INSERT INTO player_matches ('

    #parse data and insert
    for j in range(step):
        #create the query
        values = ()
        try:
            for k, v in response.json()['rows'][j].iteritems():
                if k=='chat' or k=='picks_bans' or k=='objectives' or\
                    k=='radiant_gold_adv' or k=='radiant_xp_adv' or k=='teamfights'or k=='cosmetics':
                    continue
                if j==0:
                    insert_query+=' '+k+','

                values = values + (v,)
        except Exception as e:
            #print '1',e
            pass
        tuples.append(values)
            
    insert_query = insert_query.rstrip(',')
    insert_query += ')' + ' values '

    try:
        args_str = ','.join(cur.mogrify('(%s, %s, %s, %s)',x)\
            for x in tuples)
        cur.execute(insert_query + args_str + 'on conflict  (match_id, player_slot) DO NOTHING')
    except Exception as e:
        print '2',e
        pass
myConnection.commit()
myConnection.close()    
    

In [29]:
#%%time
#cols to obtain: "match_id":17955123,"account_id":89782335,"player_slot":0,"hero_id":31,
#use requests to get data
endpoint='https://api.opendota.com/api/explorer?sql='
max_records = 120
step = 120

query="select%20*%20from%20heroes"

myConnection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
cur = myConnection.cursor()

offset = i * step
url = endpoint+query
start = time.time()
response = requests.get(url)
end = time.time()
response.json()['rows']

{u'attack_type': u'Ranged',
 u'id': 21,
 u'legs': 2,
 u'localized_name': u'Windranger',
 u'name': u'npc_dota_hero_windrunner',
 u'primary_attr': u'int',
 u'roles': [u'Carry', u'Support', u'Disabler', u'Escape', u'Nuker']}

# Grab hero data

In [43]:
#%%time
#cols to obtain: "match_id":17955123,"account_id":89782335,"player_slot":0,"hero_id":31,
#use requests to get data
endpoint='https://api.opendota.com/api/explorer?sql='
max_records = 120
step = 120

query="select * from heroes"

myConnection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
cur = myConnection.cursor()

#get 1 records at a time
for i in log_progress(range(max_records/step)):
    #print i, 'out of', max_records/step, 
    offset = i * step
    url = endpoint+query
    start = time.time()
    response = requests.get(url)
    end = time.time()
    tuples = []
    insert_query = 'INSERT INTO heroes ('

    #parse data and insert
    for j in range(step):
        #create the query
        values = ()
        try:
            for k, v in response.json()['rows'][j].iteritems():
                if k=='legs':
                    continue
                if j==0:
                    insert_query+=' '+k+','

                values = values + (v,)
            tuples.append(values)
        except Exception as e:
            #print '1',e, j
            pass
       
            
    insert_query = insert_query.rstrip(',')
    insert_query += ')' + ' values '
    try:
        args_str = ','.join(cur.mogrify('(%s, %s, %s, %s, %s, %s)',x)\
            for x in tuples)
        cur.execute(insert_query + args_str + "on conflict (id) DO NOTHING" )
    except Exception as e:
        print '2',e, j
        pass
myConnection.commit()
myConnection.close()    
    

# Player Ratings

no privilege to look at this table :(


In [ ]:
#%%time
#cols to obtain: "match_id":17955123,"account_id":89782335,"player_slot":0,"hero_id":31,
#use requests to get data
endpoint='https://api.opendota.com/api/explorer?sql='
max_records = 480000
step = 400

query='select * from player_ratings LIMIT ' + str(step) + ' Offset '

myConnection = psycopg2.connect(host=hostname, user=username, password=password, dbname=database)
cur = myConnection.cursor()

#get 1 records at a time
for i in log_progress(range(max_records/step)):
    #print i, 'out of', max_records/step, 
    offset = i * step
    url = endpoint+query+str(offset)
    start = time.time()
    response = requests.get(url)
    end = time.time()
    tuples = []
    insert_query = 'INSERT INTO player_ratings ('

    #parse data and insert
    for j in range(step):
        #create the query
        values = ()
        try:
            for k, v in response.json()['rows'][j].iteritems():
                if k=='chat' or k=='picks_bans' or k=='objectives' or\
                    k=='radiant_gold_adv' or k=='radiant_xp_adv' or k=='teamfights'or k=='cosmetics':
                    continue
                if j==0:
                    insert_query+=' '+k+','

                values = values + (v,)
        except Exception as e:
            print '1',e
            pass
        tuples.append(values)
            
    insert_query = insert_query.rstrip(',')
    insert_query += ')' + ' values '

    try:
        args_str = ','.join(cur.mogrify('(%s, %s, %s, %s)',x)\
            for x in tuples)
        cur.execute(insert_query + args_str + 'on conflict  (match_id, player_slot) DO NOTHING')
    except Exception as e:
        print '2',e
        pass
myConnection.commit()
myConnection.close()    
    

In [ ]:
player_ratings